In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates

df =  pd.read_csv('dataset.csv')
df.sort_values(by = 'Date', inplace=True)
df.reset_index(inplace=True)
df.drop(labels = 'index',axis =1, inplace=True)
df['Data_Value']= df['Data_Value'].apply(lambda x:  x/10)

In [ ]:
df_min = df[df['Element'] == 'TMIN']
df_min.drop(labels = 'Element', inplace=True, axis= 1)
df_min = df_min.groupby('Date').agg(min)

In [ ]:
df_max = df[df['Element'] == 'TMAX']
df_max.drop(labels = 'Element', inplace=True, axis=1)
df_max = df_max.groupby('Date').agg(max)

In [ ]:
df_max = df_max.merge(df_min, how = 'inner', left_index=True, right_index=True)

In [ ]:
df_max.drop(labels=['ID_x','ID_y'], inplace=True, axis=1)
labels = ['T_max','T_min']
df_max.columns = labels

In [ ]:
# make the index as datetime format
df_max.index = pd.to_datetime(df_max.index) 

In [ ]:
df_lineplot=df_max.loc['2005':'2014'].copy(deep =True) #filters out everything after 2014
df_lineplot = df_lineplot.loc[~((df_lineplot.index.month == 2) & (df_lineplot.index.day == 29))] ##remove leap

In [ ]:
df_lineplot.index = df_lineplot.index.map(lambda t: t.replace(year=1999))
df = df_lineplot
df['date']=pd.to_datetime(df.index)
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df_grouped=df.pivot_table(index=('month','day'),values=['T_max','T_min'],aggfunc=['max','min'])
df_grouped2 = pd.concat([df_grouped['max','T_max'], df_grouped['min','T_min']], axis=1)

In [ ]:
series1 = df_grouped['max','T_max']
series1.reset_index(inplace=True,drop=True)
series1.index = pd.date_range(start='1999-01-01',end='1999-12-31',freq='D')
series2 = df_grouped['min','T_min']
series2.reset_index(inplace=True,drop=True)
series2.index = pd.date_range(start='1999-01-01',end='1999-12-31',freq='D')

In [ ]:
plt.figure()
plt.plot(series1,'-',linewidth=.5,color='black')
plt.plot(series2,'-', linewidth=.5, color='black')
plt.box(on=None)

In [ ]:
plt.gca().fill_between(series1.index,series2,series1,alpha =.2,color = '#A7F158')
#specify x-axis (index in this case), and the two curves


In [ ]:
plt.title('Record High and Low Temperature Observations')
plt.xticks(ticks = (pd.date_range(start='1999-01-01',end='1999-12-31',freq='3M')), labels=['31-jan','30-apr','31-jul','31-oct'])

In [ ]:
plt.gca().margins(0.0) #stretch to the ends
plt.ylabel('Temperature $[°C]$')
plt.xlabel('Date')

In [ ]:
df_2015 = df_max.loc['2015'].copy(deep = True)
df_2015.head()

In [ ]:
df_2015['date']=pd.to_datetime(df_2015.index)
df_2015['year']=df_2015['date'].dt.year
df_2015['month']=df_2015['date'].dt.month
df_2015['day']=df_2015['date'].dt.day
df_2015=df_2015.pivot_table(index=('month','day'),values=['T_max','T_min'],aggfunc=['max','min'])
df_2015 = pd.concat([df_2015['max','T_max'], df_2015['min','T_min']], axis=1)

In [ ]:
df_2015 =df_2015.merge(df_grouped2, right_index=True,left_index=True)
df_2015.columns = ['max 15','min 15','max 05-14', 'min 05-14']

In [ ]:
df_final = df_2015[(df_2015['max 15'] > df_2015['max 05-14']) | (df_2015['min 15'] < df_2015['min 05-14'])]
df_final.head()

In [ ]:
rec_high_15 = df_final[df_final['max 15'] > df_final['max 05-14']]
rec_high_15 =  rec_high_15['max 15']
rec_high_15 = pd.concat({'1999': rec_high_15}, names=['Year'])
rec_high_15 =rec_high_15.to_frame()

In [ ]:
rec_high_15['date']=(rec_high_15.index)
rec_high_15.reset_index(inplace=True)
rec_high_15.index = pd.to_datetime(rec_high_15[['Year','month','day']]) 

In [ ]:
plt.plot(rec_high_15['max 15'],'+',color ='red',markersize =10)
plt.legend(('Record maximum for 2015',))

In [ ]:
rec_low_15 = df_final[df_final['min 15'] < df_final['min 05-14']]
rec_low_15 = rec_low_15['min 15']
rec_low_15 = pd.concat({'1999': rec_low_15}, names=['Year'])
rec_low_15 =rec_low_15.to_frame()
rec_low_15['date']=(rec_low_15.index)
rec_low_15.reset_index(inplace=True)

In [ ]:
rec_low_15.index = pd.to_datetime(rec_low_15[['Year','month','day']]) 
plt.plot(rec_low_15['min 15'],'+',color ='blue',markersize =10)

In [ ]:
from matplotlib.font_manager import FontProperties

fontP = FontProperties()
fontP.set_size('x-small')
plt.legend(('Record high and low for 2005-2014','_nolegend_','Record max in 2015','Record min in 2015'),bbox_to_anchor=(0.9,0), loc='lower right',prop=fontP)
